# Introduction

Citibike is the leading bikesharing platform in New York, including the districts of Manhattan, Brooklyn, Queens & Jersey City. That said, the company is far from being profitable due to growing competition from other modes of transportation (e.g., scooters, electric bikes, etc.). According to a consulting firm hired a few months ago, the operations of Citibike are inefficient due to the lack of data-driven decision processes. If Citibike were only 4\% more efficient, it would become profitable. During peak hours (e.g., morning commute time), the bikesharing platform is particularly unreliable. Approximately 20\% of the dock stations face a "stock-out" situation, meaning that there are no bikes available. In such cases, commuters choose other modes of transportation. 

<img src="citi-bike.jpg" width="500">

### Your assignment
The consulting firm recommends using **optimization models** to improve the operations, especially during peak time. As part of this new strategy, you are recently hired as a lead data scientist of Citibike. You suspect that the inventory of bikes and the dock stations are not well positioned given the incoming demand at peak times. In this workshop, you will develop a data-driven optimization model to improve the positioning of inventory at peak time. In particular, you will formulate and implement integer programming models.

*Workshop instructions*: **Please carefully read all the instructions below:**
- The code cells are partially filled. You need to complete and execute the code in each cell. *You can consult any material posted on Canvas to help you get started with the syntax. You can ask questions to the TAs and discuss with your teammates.*
- Make sure to answer all the questions along the way in a detailed manner.
- This is an individual assignment; you should differentiate your coding style, answers, commenting, etc. from those of your teammates.
- Only Q1-Q3.6 are meant to be covered in class. You are free to start working on the other questions, but you should not exchange with your teammates on all remaining questions.

*Submission*: Your submission should include the following:
- *Notebook:* An html file along with a .ipynb file **(85 pt)**
- *Executive summary:* A 1 pager report summarizing all your findings (at least 11 pt font, 1 inch margin, pdf/doc format). This report should take the form of an executive summary that combines elements of your analysis and business recommendations. Supporting evidence can be provided in an appendix, or by referencing the questions of the workshop. The report will be evaluated along 3 dimensions: clarity, scientific validity, practical relevance. **(15 pt)**


# Q1. Visualize the data (15 pt)

## Load the data

Your colleague has generated data sets that contains all the needed information about the demand and available inventory of bikes during *one hour at peak time*. Your initial analysis will be based on the following data sets:

- `demand.csv`: describes the number of bikes rented at each dock station (during one peak hour),
- `starting_inventory.csv`: describes the initial inventory of bikes at each dock station (before peak hour),
- `distances.csv`: describes the distance between any two dock stations (in km unit).


Ideally, the inventory of bikes available the beginning of the peak hour should approximately match the expected amount of demand. For example, if you expect 5 user requests in station A, and 5 user requests in station B in the next hour, the number of bikes in station A and B should be roughly equal, otherwise there is an imbalance of inventory. Using spatial visualization, you will check if the initial inventory is balanced or imbalanced.

*Execute the cells below. Read carefully all the comments*

In [1]:
# Import various packages
import pandas as pd
import numpy as np
import folium # visualisation package for spatial data (plot of maps)
import seaborn as sns # general visualization package 
import matplotlib.pyplot as plt # general visualization package 
#next command allows you to display the figures in the notebook
%matplotlib inline     

In [2]:
## In case folium is not installed, run the command below and import folium again
# ! conda install -c conda-forge folium --yes

Next, you need to load the data sets as dataframes using the Pandas function `pd.read_csv()`.

In [3]:
starting_inventory = pd.read_csv("starting_inventory.csv", index_col=0) 
# "index_col=0" is used so that the dataframe is indexed by the first column, which is the dock station ID
demand = pd.read_csv("demand.csv", index_col=0)
distances = pd.read_csv("distances.csv", index_col=0)
distances.columns = list(map(lambda x: int(x),distances.columns.tolist())) # the column names (dock IDs) are converted into integers
replenishment = pd.read_csv("replenishment.csv", index_col=0)


Using the command `.describe()`, you can familiarize yourself with the format of the dataframes, and  check if there are missing entries.

In [4]:
# Insert your code here
starting_inventory.describe()

,count,latitude,longitude
count,689.000000,689.000000,689.000000
mean,30.724238,40.732527,-73.967780
std,70.308432,0.041163,0.024024
min,10.000000,40.655400,-74.025353
25%,10.000000,40.695807,-73.987030
50%,10.000000,40.728419,-73.968044
75%,19.000000,40.767100,-73.949450
max,739.000000,40.814394,-73.910651


In [5]:
demand.describe()

,latitude,longitude,count
count,689.000000,689.000000,689.000000
mean,40.732527,-73.967780,27.648766
std,0.041163,0.024024,56.377839
min,40.655400,-74.025353,1.000000
25%,40.695807,-73.987030,5.000000
50%,40.728419,-73.968044,10.000000
75%,40.767100,-73.949450,26.000000
max,40.814394,-73.910651,428.000000


In [6]:
distances.describe()

,119,120,127,143,144,146,150,157,161,167,...,534,536,539,540,545,546,72,79,82,83
count,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,...,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000,689.000000
mean,5.234788,5.818614,5.660433,5.815784,5.173995,5.896868,4.557984,6.009891,5.136558,4.691174,...,6.487351,4.529948,4.646942,4.676314,4.500302,4.725332,5.935885,5.695771,5.451519,5.886057
std,3.101918,3.240807,2.416317,3.396272,3.039626,2.765462,2.232180,3.456671,2.369193,2.011353,...,3.129782,1.880601,2.148153,2.001787,1.912251,2.026796,2.523133,2.677876,2.832379,3.535927
min,0.000000,0.000000,0.000000,0.000135,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.778453,3.166575,3.980364,2.922870,2.795046,3.792634,2.907156,3.077351,3.575707,3.018884,...,3.896037,3.009274,3.131241,3.093796,2.985554,3.137539,4.043380,3.715493,3.254939,3.078088
50%,4.243954,5.492614,5.792542,5.194791,4.199160,5.789344,4.364615,5.426828,5.290044,4.733095,...,6.257024,4.590299,4.426917,4.846375,4.599683,4.905780,5.945542,5.633804,5.191993,5.067243
75%,7.926693,8.437574,7.444620,8.712969,7.752457,7.900162,5.982674,8.953698,6.833613,6.320707,...,8.895602,6.124998,6.031318,6.364107,6.043458,6.460421,8.018556,7.575516,7.511705,8.959315
max,12.087375,12.776908,10.356681,12.864326,11.919733,11.534052,9.753680,13.111354,9.844240,9.969009,...,12.871711,9.299113,9.929280,9.222172,8.738407,9.322635,11.311183,11.157840,11.482100,13.259187


In [7]:
replenishment.describe()

,latitude,longitude,count
count,689.000000,689.000000,689.000000
mean,9.869376,-17.764877,6.513788
std,1.136245,2.045241,11.240249
min,0.000000,-18.000000,0.000000
25%,10.000000,-18.000000,1.000000
50%,10.000000,-18.000000,3.000000
75%,10.000000,-18.000000,7.000000
max,10.000000,0.000000,108.000000


### Q1.0. What is the distance between the dock stations "120" and "146"? What is the maximum inventory of bikes initially available over all dock stations?

*Insert answer here:*

The distance between the dock stations "120" and "146" is 5.79km. 

The maximum inventory of bikes initially available over all dock stations is 739.

In [8]:
distances[146][120]

5.7893437175538125

In [9]:
starting_inventory['count'].max()

739

## Plot the heatmaps of demand and inventory

In the next cells, we will generate a heatmap of New York, representing the level of demand in different regions.

*The code is provided. Feel free to experiment with different parameters to better understand their respective roles.*

In [10]:
def generateBaseMap(default_location=[40.71, -73.96], default_zoom_start=12.5):
    '''
    This function generates a base map using the folium package
    
    Arguments:
    default_location -> latitude, longitude (a list)
    default_zoom_start -> level of zoom of the map (a positive number)
    '''
    base_map = folium.Map(location=default_location,control_scale=True, zoom_start=default_zoom_start)
    # 'Stamen Toner' is a type of map; by default the map is "OpenStreetMap"
    return base_map

The next cell will plot a heatmap of the demand.

In [11]:
# Import the HeatMap object constructor from the folium package
from folium.plugins import HeatMap 

# To begin, generate a base map, which will be edited next
base_map = generateBaseMap()

# The demand is converted into a list of t-uples, each describing one request point: (longitude, latitude, count)
# Copy the demand dataframe 
demand_agg = demand.copy()
# Rounding latitudes and longitudes to nearest 0.001 decimal
demand_agg.loc[:,['latitude', 'longitude']] = demand_agg.loc[:,['latitude', 'longitude']].round(3)
# Aggregating by latitude and longitude 
demand_agg= demand_agg[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum()
# Converting demand into a list
demand_list = demand_agg.reset_index().values.tolist()

# Add the desired heatmap to the base map
HeatMap(data= demand_list,radius=0, max_zoom=15,minOpacity=0).add_to(base_map)
base_map

Ideally, you would like to compare the expected demand to the initial inventory at the dock stations. To this end, you need to visualize the inventory of bikes initially available.

*By re-using the code above, construct a heatmap for the inventory of bikes in the city (`starting_inventory`). You need to edit the lines of code relative to the construction of the demand list.*

In [12]:
# Uncomment and fill the code below

from folium.plugins import HeatMap
base_map = generateBaseMap()
inventory_agg = starting_inventory.copy()
# Create a line of code to round latitudes and longitudes
inventory_agg.loc[:,['latitude', 'longitude']] = inventory_agg.loc[:,['latitude', 'longitude']].round(3)
# Create a line of code to aggregate the inventory by latitudes and longitudes
inventory_agg= inventory_agg[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum()
# Convert inventory_agg to a list inventory_list (fill below)
inventory_list = inventory_agg.reset_index().values.tolist()
HeatMap(data= inventory_list,radius=0, max_zoom=15,minOpacity=0).add_to(base_map)
base_map

### Q1.1. Are there apparent mismatches between demand and inventory as you compare the two heatmaps?

*Insert answer here:* 

We can see that there are clear under-supply at the intersection of 6th Avenue and 34th Street; the block around Duane Street and Reade Street; the intersection of 23rd Street and 7th Avenue; and the intersection of 23rd Street and 7th Avenue.
However, it is not visually intuitive to say which areas have excess supply. 

Your colleagues argue that that it is difficult to eyeball the differences between the two heatmaps. It would be preferable to have a single heatmap showing the "imbalance" between demand and inventory. Specifically, you would like to identify  areas where the initial bike inventory is either insufficient or excessive, compared to the amount of demand.

### Q1.2. How can you quantify  the degree of imbalance between demand and inventory? Create heatmaps for the newly defined imbalance metric(s) showing the areas with lack or excess of bikes.

*Hint:* Define two new metrics that quantify the imbalance between demand inventory and reuse the same code as above: 
- The first metric could capture the deficit of inventory at the stations where demand_agg > inventory_agg,
- The second metric could capture the excess of inventory at the stations where inventory_agg > demand_agg.

*Insert answer here:*

We can use $demand\_agg / (inventory\_agg+1)$ to indicate the level of inventory deficit. 

We can use $inventory\_agg / (demand\_agg+1)$ to indicate the level of inventory excess. 

In [13]:
# Uncomment and fill the code below

base_map = generateBaseMap()
deficit = np.divide(demand_agg,1+inventory_agg)
deficit_list = deficit.fillna(0).reset_index().values.tolist()
HeatMap(data= deficit_list,radius=0, max_zoom=15,minOpacity=0).add_to(base_map)
base_map

In [14]:
# Uncomment and fill the code below

base_map = generateBaseMap()
excess = np.divide(inventory_agg,1+demand_agg)
excess_list = excess.fillna(0).reset_index().values.tolist()
HeatMap(data= excess_list,radius=0, max_zoom=15,minOpacity=0).add_to(base_map)
base_map

### Q1.3. Add one more visualization of your own. Describe what it shows and what your learn from it in 1-2 lines

*Insert answer here:*

Comparing replenishment heatmap and inventory excess heatmap, we can see that the replenishment is the highest within areas with inventory excess. This means that Citibike is not optimizing the replenishment according to different levels of demand.  

In [15]:
# Insert your code here
# To begin, generate a base map, which will be edited next
base_map = generateBaseMap()

# The replenishment is converted into a list of t-uples, each describing one request point: (longitude, latitude, count)
# Copy the replenishment dataframe 
replenish_agg = replenishment.copy()
# Rounding latitudes and longitudes to nearest 0.001 decimal
replenish_agg.loc[:,['latitude', 'longitude']] = replenish_agg.loc[:,['latitude', 'longitude']].round(3)
# Aggregating by latitude and longitude 
replenish_agg= replenish_agg[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum()
# Converting replenishment into a list
replenish_list = replenish_agg.reset_index().values.tolist()

# Add the desired heatmap to the base map
HeatMap(data= demand_list,radius=0, max_zoom=15,minOpacity=0).add_to(base_map)
base_map

# Q2. Optimal Rebalancing of Bike Inventory (20 pt)

**Matching supply with demand**: Given the imbalances between demand and inventory, Citibike runs frequent "rebalancing" operations, where bikes are relocated from a dock station having an excess of inventory to a station having insufficient inventory. These rebalancing operations are run in an ad-hoc fashion. You would like to construct an optimization model to rigorously rebalance the inventory before the beginning of the peak hour.

**The economics of rebalancing:** Rebalancing operators have to be paid \\$1 per km per relocated bike. There are no geographical restrictions on the rebalancing of bikes (from any station to any other station). After the rebalancing is performed, the amount of demand satisfied at each station will the minimum between the new initial inventory level and the total demand (we assume that there is no further replenishment during the peak hour). Each satisfied user request brings \\$4 of revenue, on average. Hence, when bikes are unavailable and user requests are unfulfilled, Citibike loses the revenue opportunity of \\$4 per user. 

*In what follows, you will formulate and implement the rebalancing problem as an integer program. Starting with the inventory levels of `starting_inventory`, you want to find the bikes relocations that maximize the total net revenue. For simplicity, you can assume that the rebalancing operations happen instantaneously before the beginning of the peak hour. There is no increase in the supply of bikes during the peak hour.*

In [16]:
# The following lines of code import the gurobi package
import gurobipy as gp
from gurobipy import GRB,quicksum

## Model creation

### Q2.1. Create a GUROBI new model "m", named "rebalancing"

In [17]:
#Insert your code here:
m = gp.Model('rebalancing')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-02-10


## Decision variables

Clearly your main decisions have to do with the "relocation" of bikes.

In [18]:
# Instantiate the list of all stations
stations = demand.index.tolist()

# Next, we create an "integer" decision variable for each pair of dock stations (A,B)
relocation = m.addVars(stations,stations,vtype=GRB.INTEGER, lb = 0, name="relocation")

### Q2.2. Explain in detail what the code in the cell above is doing.

*Insert your answer here:*

The first line is adding all the stations (e.g., 119, 120, etc.) into a list with list object name "stations". 

The second line is creating a decision variable as a dictionary (key value pair) with object name "relocation". This includes the key-value pair, with key in the form of {station1, station2}, and value as an integer. The value indicates how many bikes will be relocated from station1 to station2. 

Your colleague thinks that additional decision will be needed for the problem. She suggests the following line of code:

In [19]:
satisfied_demand = m.addVars(stations,vtype=GRB.INTEGER, lb = 0, name="satisfied_demand")

### Q2.3. Why are the variables `satisfied_demand` needed? How do we refer to such variables (which do not describe actual decisions)?

*Insert your answer here:*

In this case, there may be excessive supply in some stations, but only the number of bikes that are demanded will generate revenue. On the other hand, there may be excessive demand, but only limited inventory are available after re-balancing and generate revenue. Therefore, we need a new decision variable (satisfied_demand) indicating the number of matches of supply and demand incorporating both conditions. 

This is called auxiliary variable,  which is not an actual decision but helpful to model the problem. 

## Constraints

### Q2.4. Is there an upper bound on the number of bikes that can be relocated from one station to another? Incorporate the upper bound into the model.
*Hint: Use `starting_inventory.loc[i,"count"]` to access the initial number of bikes at station `i`.*

*Insert your answer here:*

There should be an upper bound of number on the number of bikes that can be relocated from one station to another. The upper bound is the starting inventory in each stations.

In [20]:
#Insert your code here:
for s1 in stations:    
    m.addConstr(quicksum(relocation[s1, s2] for s2 in stations) <= starting_inventory.loc[s1,"count"]);

The satisfied demand in each station is the minimum between the demand and the inventory after rebalancing. For example, if station A has 10 bikes before the relocation and we relocate 2 extra bikes, we can satisfy up to 12 users. If the demand at station A is 8, we satisfy all 8 users. If the demand at station A is 15, we can only satisfy 12 users, and we lose 3 user requests. In general, we have the equation:

$$ {\rm satisfiedDemand}(A) ={\rm minimum} \left\{ {\rm InventoryAfterRebalancing}(A), {\rm Demand}(A)\right\}$$

Below, you will add two ensembles of constraints to our model `m` to capture the notion of satisfied demand.

### Q2.5. Add constraints imposing that `satisfied_demand` is smaller or equal to the demand, in each station.

In [21]:
#Insert your code here:
m.addConstrs(satisfied_demand[s1] <= demand.loc[s1, "count"] for s1 in stations);

### Q2.6. Explain what the additional constraints below on the `satisfied_demand`  are doing. Why they are needed?

*Insert your answer here:*

For each station, the satisfied demand should be less or equal to the number of bikes after rebalancing. The RHS denotes the number of bikes after rebalanding (starting_inventory + inflow - outflow), which can be expressed as:

$ satisfiedDemand_i \leq startingInventory_i + \sum_j relocation_{j,i} - \sum_j relocation_{i,j} $

In [22]:
# Additional constraints
m.addConstrs((satisfied_demand[i] <= starting_inventory.loc[i,"count"] \
              + quicksum(relocation[j,i] for j in stations) -quicksum(relocation[i,j] for j in stations)
                  for i in stations),name = "satisfied_demand_vs_inventory_after_rebalancing");

## Objective

### Q2.7. How would you formulate the net revenue as a linear expression? Specify the objective of the model.

*Hint: Recall that the objective function is specified using:* `m.setObjective(EXPRESSION,GRB.MAXIMIZE)`

*Insert your answer here:* 

The net revenue in this case is total revenue (number of bikes satisfied demand * revenue per bike) minus total relocation cost (distance * number of bikes relocated). This can be denoted as follows:

$\max_{relocation_{i,j} \geq 0} \quad\quad\quad  4 \cdot \sum_i satisfiedDemand_i - 1 \cdot \sum_{i,j} distance_{i,j} \cdot relocation_{i,j} $

In [23]:
#Insert your code here:
m.setObjective(4 * satisfied_demand.sum() - quicksum(1 * distances.loc[s1, s2] * relocation[s1, s2] for s1 in stations for s2 in stations), GRB.MAXIMIZE)

## Solve

Congratulations! You have formulated and implemented the integer program. You can now optimize the rebalancing and printout the optimal revenue/

In [24]:
# Run the optimization
# Note: it is not convenient to printout the relocation solution. We will develop a suitable visualization tool.
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nNet revenue: %g' % m.objVal)
    else:
        print('No solution:', m.status)
        
m.optimize()
printSolution()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 2067 rows, 475410 columns and 1424163 nonzeros
Model fingerprint: 0x705b3843
Variable types: 0 continuous, 475410 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-04, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective 62468.000000
Presolve removed 689 rows and 19978 columns
Presolve time: 3.38s
Presolved: 1378 rows, 455432 columns, 1337358 nonzeros
Variable types: 0 continuous, 455432 integer (6671 binary)
Found heuristic solution: objective 62739.189353

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity     -7.5668248e+04      4s
       1        787   2.2867491e+08  -7.4763233e+04      4s
       2       2007  -7.1147234e+04

### Q2.8. Read the output of the optimization: What is the optimal net revenue?

*Insert your answer here:* 

The optimal net revenue is $71314.1.

### Q2.9. How much revenue does Citibike gain using the optimal rebalancing compared to no rebalancing? Is this a significant increase? Carefully explain how you obtain the answer (feel free to add code below).
*Hint: The goal here is to quantify how the revenue changes after the optimal rebalancing vs. no rebalancing. To compute the revenue in the absence of rebalancing, you could reuse our optimization model with a small modification of the variables `relocation`.*

The relocations are visualized on the heatmap in the next cell. 

*Code is provided, Execute the next cells. Feel free to vary the parameters to understand their role.*

In [25]:
from collections import namedtuple
def get_bearing(p1, p2):
    
    '''
    Returns compass bearing from p1 to p2
    
    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    Return
    compass bearing of type float
    
    Notes
    Based on https://gist.github.com/jeromer/2005586
    '''
    
    long_diff = np.radians(p2.lon - p1.lon)
    
    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    bearing = np.degrees(np.arctan2(x, y))
    
    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing
def get_arrows(locations, color='black', size=4, n_arrows=3,opacity = 1):
    
    '''
    Get a list of correctly placed and rotated 
    arrows/markers to be plotted
    
    Parameters
    locations : list of lists of lat lons that represent the 
                start and end of the line. 
                eg [[41.1132, -96.1993],[41.3810, -95.8021]]
    arrow_color : default is 'blue'
    size : default is 6
    n_arrows : number of arrows to create.  default is 3
    Return
    list of arrows/markers
    '''
    
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    
    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90
    
    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows+1]
    
    arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, color = color, number_of_sides=3, 
                      radius=size, rotation=rotation,opacity = opacity))
    return arrows

base_map = generateBaseMap() # generates our base map of NY
# Next we compute the imbalance between demand and inventory
imbalance = np.divide(demand_agg,1+inventory_agg)
imbalance_list = imbalance.fillna(0).reset_index().values.tolist()


# The next loop plots the relocation lines between any two stations.
# The more opaque is the line, the more relocations are made
for i in stations:
    for j in stations:
        if relocation[i,j].x >0:
            p1 = [demand.loc[i,"latitude"],demand.loc[i,"longitude"]]
            p2 = [demand.loc[j,"latitude"],demand.loc[j,"longitude"]]
            opacity = relocation[i,j].x/starting_inventory.loc[i,"count"] +relocation[i,j].x/20
            folium.PolyLine(locations=[p1, p2], color='black', opacity = opacity).add_to(base_map)
            arrows = get_arrows(locations=[p1, p2],color='black', n_arrows=1,opacity = opacity)
            for arrow in arrows:
                arrow.add_to(base_map)

#Plots the heatmap
HeatMap(data= imbalance_list,radius=0,max_zoom=15,minOpacity=0).add_to(base_map)

#Displays the map
base_map       

In [26]:
# if no rebalancing 
m1 = gp.Model('no rebalancing')

# Instantiate the list of all stations
stations = demand.index.tolist()

relocation = m1.addVars(stations,stations,vtype=GRB.INTEGER, lb = 0, ub = 0, name="relocation")

satisfied_demand = m1.addVars(stations,vtype=GRB.INTEGER, lb = 0, name="satisfied_demand")

m1.addConstrs(satisfied_demand[s1] <= demand.loc[s1, "count"] for s1 in stations);

m1.addConstrs((satisfied_demand[i] <= starting_inventory.loc[i,"count"] \
              + quicksum(relocation[j,i] for j in stations) -quicksum(relocation[i,j] for j in stations)
                  for i in stations),name = "satisfied_demand_vs_inventory_after_rebalancing");

m1.setObjective(4 * satisfied_demand.sum() - quicksum(1 * distances.loc[s1, s2] * relocation[s1, s2] for s1 in stations for s2 in stations), GRB.MAXIMIZE)

# Run the optimization
def printSolution():
    if m1.status == GRB.OPTIMAL:
        print('\nNet revenue: %g' % m1.objVal)
    else:
        print('No solution:', m1.status)
        
m1.optimize()
printSolution()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1378 rows, 475410 columns and 949442 nonzeros
Model fingerprint: 0xc88d1692
Variable types: 0 continuous, 475410 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-04, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective 62468.000000
Presolve removed 1378 rows and 475410 columns
Presolve time: 0.12s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.22 seconds (0.19 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 62468 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.246800000000e+04, best bound 6.246800000000e+04, gap 0.0000%

Net revenue: 62468


*Insert your answer here:*

Comparing the result of net revenue with no rebalancing (\\$62468) and result with rebalancing (\\$71314.1), we can see that rebalancing causes net revenue to increase by 14.2%. This is a significant increase.

The approach is to reuse the previous model, with a tweak on "relocation" variable. Since in this case, no relocation is allowed, $relocation_{i,j}$ = 0. Therefore, by making lb=0 and ub=0, the alteration is achieved. 

### Q2.10. How do you interpret the plot? Is this what you would expect?

*Insert your answer:* 

We can see from the above plot that in general, the rebalancing routes are from areas of excessive inventory towards the nearest areas with excessive demand. For example, bikes are relocated from west Brooklyn to east Manhattan. This is expected because 1) it is profitable to satisfy the excess demand by increasing the inventory level via relocating; 2) it is cost-saving to transfer excessive bikes to the nearest areas in need, instead of further areas.    

# Q3. Rebalancing with replenishment (20 pt)

You manager has a concern about your analysis in Q2:

*I like the idea of rebalancing bikes based on an optimization model. However, you have omitted an important aspect of the problem. When a user completes her current trip, she returns the bike to the system. A new bike is made available at the destination station. You don't account for this in your current inventory, but this organic replenishment of bikes might be very helpful since it increases the number of bikes available at the destination station!*

The goal of Q3 is to update the model accordingly. You will reuse most of the existing code, with minor modifications.  Your analysis will be based on the following additional data set:

- `replenishment.csv`: describes the number of extra bikes that will be made available at each station after current users complete their trips. 

**For simplicity, we will assume that the replenishment happens instantaneously before the beginning of the peak hour.**

### Q3.1. Based on your manager's comment, did you under-estimate or over-estimate the inventory at each station in the previous question Q2?

*Insert your answer here:* 

Based on the manager's comment, I under-estimated the inventory at each station. Adjusting to his comment, the inventory level should be expressed as:

$inventory\_level = starting\_inventory + organic\_replenishment + inflow - outflow$

## Loading the data

In [27]:
replenishment = pd.read_csv("replenishment.csv", index_col=0)

*You can visualize the replenishment data using `.describe()`.*

In [28]:
replenishment.describe()

,latitude,longitude,count
count,689.000000,689.000000,689.000000
mean,9.869376,-17.764877,6.513788
std,1.136245,2.045241,11.240249
min,0.000000,-18.000000,0.000000
25%,10.000000,-18.000000,1.000000
50%,10.000000,-18.000000,3.000000
75%,10.000000,-18.000000,7.000000
max,10.000000,0.000000,108.000000


## Model creation

### Q3.2 Create the model object

*Hint: Reuse the same code as Q2.1.*

In [29]:
#Insert your code here:
m = gp.Model('rebalancing-replenishment')

## Decision variables

### Q3.2. Add the decision variables to `m`
*Hint: Reuse the same code as Q2.2 and Q2.3*

In [30]:
#Insert your code here:
relocation = m.addVars(stations, stations, vtype = GRB.INTEGER, lb = 0, name = 'relocation')
satisfied_demand = m.addVars(stations, vtype = GRB.INTEGER, lb = 0, name = 'satisfied_demand')

## Constraints

### Q3.3. Add an upper bound on the number of bikes that can be relocated from one station to another.
*Hint: Reuse the same code as Q2.4*

In [31]:
#Insert your code here:
m.addConstrs((quicksum(relocation[s1, s2] for s2 in stations) <= starting_inventory.loc[s1, 'count'] for s1 in stations));

### Q3.4. Add constraints requiring that `satisfied_demand` is smaller or equal to the demand.
*Hint: Reuse the same code as Q2.5*

In [32]:
#Insert your code here:
m.addConstrs(satisfied_demand[s1] <= demand.loc[s1, "count"] for s1 in stations);

 ### Q3.5. The `satisfied_demand` should also be related to inventory. How would you modify the constraint of Q2.6 to account for replenishment? Add these constraints to the model.
 
 *Hint: You should use the same constraint as Q2.6, but also incorporate the replenishment quantity. For simplicity, you can assume that all the replenishment happens before the demand arrives. This assumption is reasonable if the system is "stationary".*

*Insert your answer here:*

For each station, the satisfied demand should be less or equal to the number of bikes after rebalancing and replenishment. The RHS denotes the number of bikes after rebalanding and replenishment (starting_inventory + inflow - outflow + replenishment), which can be expressed as:

$ satisfiedDemand_i \leq startingInventory_i + \sum_j relocation_{j,i} - \sum_j relocation_{i,j} + replenishment_i $

In [33]:
#Insert your code here:
m.addConstrs((satisfied_demand[i] <= starting_inventory.loc[i,"count"] \
              + quicksum(relocation[j,i] for j in stations) -quicksum(relocation[i,j] for j in stations) + replenishment.loc[i,"count"]
                  for i in stations),name = "satisfied_demand_vs_inventory_after_rebalancing_and_replenishment");

## Objective

### Q3.6. What is the objective function? Specify the objective of the model `m`.

*Hint: Reuse the code of Q2.7*

In [34]:
#Insert your code here:
m.setObjective(4 * satisfied_demand.sum() - quicksum(1 * distances.loc[s1, s2] * relocation[s1, s2] for s1 in stations for s2 in stations), GRB.MAXIMIZE)

*Insert your answer here:* 

The net revenue in this case is total revenue (number of bikes satisfied demand * revenue per bike) minus total relocation cost (distance * number of bikes relocated). This can be calculated as follows:

$\max_{relocation_{i,j} \geq 0} \quad\quad\quad  4 \cdot \sum_i satisfiedDemand_i - 1 \cdot \sum_{i,j} distance_{i,j} \cdot relocation_{i,j} $

## Solve

Congratulations! You have formulated and implemented the integer program. You can now optimize the rebalancing and printout the optimal revenue.

In [35]:
# Run the optimization
# Note: it is not convenient to printout the relocation solution. We will develop a suitable visualization tool.
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nNet revenue: %g' % m.objVal)
    else:
        print('No solution:', m.status)
        
m.optimize()
printSolution()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 2067 rows, 475410 columns and 1424163 nonzeros
Model fingerprint: 0x5cf05495
Variable types: 0 continuous, 475410 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-04, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+02]
Found heuristic solution: objective 72024.000000
Presolve removed 722 rows and 35401 columns
Presolve time: 3.36s
Presolved: 1345 rows, 440009 columns, 1281864 nonzeros
Variable types: 0 continuous, 440009 integer (3985 binary)
Found heuristic solution: objective 72164.331558

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity     -7.6042840e+04      4s
       1        207   8.6743383e+06  -7.5937255e+04      4s
       2        445  -7.5662779e+04

### Q3.7. Read the output of the optimization: What is the optimal net revenue? Is it greater or smaller than that obtained without replenishment in question Q.2.8? Why?

*Insert your answer here:* 

The optimal net revenue is (\\$75662.9). This is 6.1\% greater than the result obtained without replenishment (\\$71314.1). This can be because of two reasons. 

1. Some central areas with excess demand is not fully satisfied, and these areas tend to be far away from cheap access to inventory (distance too big from areas with excess supply). With free replenishment, these excess demand is satisfied. This can bring up the revenue. 

2. Replenishment of bikes is essentially free, which is cheaper than relocation. Therefore, some areas can satisfy excess demand with the number of replenishment rather than requesting relocation. This can bring down the cost. 

### Q3.8. How much revenue does Citibike gain using the optimal rebalancing compared to no rebalancing? How does this compare to Q2.9? Why?
*Hint: The goal here is to quantity the revenue change after vs. before the optimal rebalancing. To compute the revenue in the absence of rebalancing, you could reuse our optimization model with a very small modification of the variables `relocation`.*

*Insert your answer here:*

We can see that using rebalancing and replenishment will generate there is a 21.1% higher net revenue (at \\$75662.9) than no-rebalancing (\\$62468). 

The relocations are visualized on the heatmap in the next cell. *Execute the next cell.*

In [36]:
base_map = generateBaseMap() # generates our base map of NY
# Next we compute the imbalance ratio between demand and inventory
imbalance = np.divide(demand_agg,1+inventory_agg)
imbalance_list = imbalance.fillna(0).reset_index().values.tolist()


# The next loop plots the relocation lines between any two stations.
# The more opaque is the line, the more relocations are made
for i in stations:
    for j in stations:
        if relocation[i,j].x >0:
            p1 = [demand.loc[i,"latitude"],demand.loc[i,"longitude"]]
            p2 = [demand.loc[j,"latitude"],demand.loc[j,"longitude"]]
            opacity = relocation[i,j].x/starting_inventory.loc[i,"count"] +relocation[i,j].x/20
            folium.PolyLine(locations=[p1, p2], color='black', opacity = opacity).add_to(base_map)
            arrows = get_arrows(locations=[p1, p2],color='black', n_arrows=1,opacity = opacity)
            for arrow in arrows:
                arrow.add_to(base_map)

#Plots the heatmap
HeatMap(data= imbalance_list, radius=0,max_zoom=15,minOpacity=0).add_to(base_map)

#Displays the map
base_map                

### Q3.9. How do you interpret the plot? Is this what you would expect compared to Q2.10?

*Insert your answer:* 

We can see that the major differences between replenishment&rebalancing versus only rebalancing lie in 

1. the reduction in numbers of relocation routes;
2. the reduction in average distance of relocation routes; and
3. the reduction in areas involved in relocation.

In addition, there is similarity between replenishment&rebalancing and only rebalancing, which is most relocation happens from excess supply areas to excess demand areas. For example, from West Side Yard to Lincoln Tunnel. 

This is expected because as analyzed in 3.7, some central areas (e.g. 1st Avenue) with excess demand is not fully satisfied, and  With free replenishment combined with nearby relocation, these excess demand is satisfied. Therefore, these areas needs less long-distance relocation from West Brooklyn, which is costly. 

And since there are free replenishment to satisfy the excess demand in some areas, e.g., Wythe Avenue in West Brooklyn, these areas needs few relocation inflow due to its implied cost. Therefore, the areas involved in relocation is reduced. 

# Q4. Vehicle Capacity (10 pt)

In reality, Citibike can utilize two types of vehicles:
- Large capacity vehicles (LCV): These are the default vehicles. They can carry any arbitrary number of bikes. Due to the large size of the vehicle, rebalancing operators have to be paid \\$1 per km per relocated bike (similar to Q2 and Q3).
- Small capacity vehicles (SCV): These specialized vehicles can handle up to 10 bikes per rebalancing operation. Rebalancing operators have to be paid \\$0.5 per km per relocated bike.

For each relocation operation (corresponding to a pair of stations), the firm needs to choose exactly one type of vehicle. The goal of Q4 is to update the model constructed in Q3 to incorporate the following logical condition: *EITHER we use a small capacity vehicle and the number of relocated bikes is less than 10 OR we use a large capcity vehicle*.

## Decision variables: Small vs. large vehicles

We add relocation variables corresponding to the use of small vs. large vehicles.

In [37]:
# The decision variables describe the number of bikes relocated from station A to station B
relocation_SCV = m.addVars(stations,stations,vtype=GRB.INTEGER, lb = 0, name="relocation_small")
relocation_LCV  = m.addVars(stations,stations,vtype=GRB.INTEGER, lb = 0, name="relocation_large")

## Constraints

In the next constraint, we impose that the total number of relocations is the sum of relocations using SCVs and LCVs.

In [38]:
m.addConstrs(relocation[i,j] == relocation_SCV[i,j] +relocation_LCV[i,j] for i in stations for j in stations);

### Q4.1. Add constraints requiring that EITHER we use an SCV with at most 10 bikes OR we don't use an SCV 
*Hint: Use the big-M method covered in today's class. Define a binary auxiliary variable for each pair of stations to switch between the two conditions.*

In [39]:
# Uncomment and fill the code below:

M = 1e4
auxiliary = m.addVars(stations,stations,vtype=GRB.BINARY, lb=0, name="relocate 10- SCV or 10+ LCV")
m.addConstrs(relocation_LCV[i,j] <= 0 + M * auxiliary[i, j] for i in stations for j in stations);
m.addConstrs(relocation_LCV[i,j] >= 10 - M * (1 - auxiliary[i, j]) for i in stations for j in stations);

m.addConstrs(relocation_SCV[i,j] <= 10 + M * auxiliary[i, j] for i in stations for j in stations);
m.addConstrs(relocation_SCV[i,j] <= 0 + M * (1 - auxiliary[i, j]) for i in stations for j in stations);

## Objective

The objective function is modified accordingly. *Execute the cells below.*

In [40]:
# Set the objective: maximize the net revenue
m.setObjective(4*satisfied_demand.sum()
               -1*quicksum(relocation_LCV[i,j]*distances.loc[i,j] for i in stations for j in stations)
               -0.5*quicksum(relocation_SCV[i,j]*distances.loc[i,j] for i in stations for j in stations)               
               ,GRB.MAXIMIZE)

## Solve

Congratulations! You have formulated and implemented the integer program. You can now optimize the rebalancing and printout the optimal revenue.

In [41]:
# Run the optimization
# Note: it is not convenient to printout the relocation solution. We will develop a suitable visualization tool.
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nNet revenue: %g' % m.objVal)
    else:
        print('No solution:', m.status)
        
m.optimize()
printSolution()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 2375672 rows, 1899573 columns and 6646094 nonzeros
Model fingerprint: 0x21f75fb3
Variable types: 0 continuous, 1899573 integer (474721 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [7e-05, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]

MIP start from previous solve produced solution with objective 75752.5 (1.39s)
Loaded MIP start from previous solve with objective 75752.5
Processed MIP start in 1.46 seconds (2.22 work units)

Presolve removed 1642295 rows and 1074940 columns (presolve time = 5s) ...
Presolve removed 1677431 rows and 1099129 columns (presolve time = 10s) ...
Presolve removed 1851661 rows and 1101976 columns (presolve time = 15s) ...
Presolve removed 1851637 rows and 1101952 columns
Presolve time: 15.17s
Presolved: 524035 rows, 797621 columns, 2702844 

The relocations are visualized on the heatmap in the next cell. *Execute the next cell.*

In [42]:
base_map = generateBaseMap() # generates our base map of NY
# Next we compute the imbalance between demand and inventory
imbalance = np.divide(demand_agg,1+inventory_agg)
imbalance_list = imbalance.fillna(0).reset_index().values.tolist()


# The next loop plots the relocation lines between any two stations.
# The more opaque is the line, the more relocations are made
for i in stations:
    for j in stations:
        if relocation[i,j].x >0:
            p1 = [demand.loc[i,"latitude"],demand.loc[i,"longitude"]]
            p2 = [demand.loc[j,"latitude"],demand.loc[j,"longitude"]]
            opacity = relocation[i,j].x/starting_inventory.loc[i,"count"] +relocation[i,j].x/20
            folium.PolyLine(locations=[p1, p2], color='black', opacity = opacity).add_to(base_map)
            arrows = get_arrows(locations=[p1, p2],color='black', n_arrows=1,opacity = opacity)
            for arrow in arrows:
                arrow.add_to(base_map)

#Plots the heatmap
HeatMap(data= imbalance_list, radius=0,max_zoom=15,minOpacity=0).add_to(base_map)

#Displays the map
base_map                

### Q4.2. How do you interpret the output compared to Q3.7-Q3.9? Does the net revenue increase or decrease? 

*Insert your answer:* 

We can see that the net revenue after introducing SCV and LCV (\\$75881.9) is 2.9\% higher than before (\\$75662.9). The increasing net revenue is because more excess demand is satisfied because of reduced average relocation cost per unit. 

Comparing this graph against the graph in 3.8, we can see that there seems to be more long-distance relocation in the areas around Midtown. This is because the unit cost is reduced from \\$1 per bike per km to \\$0.5 per bike per km when relocating bikes using SCV (relocation <= 10). This will make the areas far but with small amount of bikes to relocate more appealing for areas with excess demand because of cheaper unit cost. 

# Q5. Procurement of vehicles (5pt)

In reality, Citibike does not operate the SCVs and LCVs, which are booked through an external provider. There are various costs and constraints associated with the procurement of  small and large vehicles. Specifically, we have
- *Fixed cost for small vehicles:* You should pay a fixed cost of 80\$ to be able to use small vehicles. Note that this cost does **not** scale with the number of small vehicles.
- *Per vehicle cost:* You should pay a variable cost of 10\$ per vehicle.
- *Number of rebalancing operations per vehicle:* Suppose that each vehicle can perform at most 3 distinct rebalancing operations witin the imparted time.

How would you modify the model to incorporate procurement costs? **Build a new model that captures all the above requirements.**

In [43]:
# build a new model
m = gp.Model('vehicle procurement');

stations = demand.index.tolist();
##################### reuse the previous model decision variables ################
relocation = m.addVars(stations, stations, vtype = GRB.INTEGER, lb = 0, name = 'relocation');
satisfied_demand = m.addVars(stations, vtype = GRB.INTEGER, lb = 0, name = 'satisfied_demand');
relocation_SCV = m.addVars(stations,stations,vtype=GRB.INTEGER, lb = 0, name="relocation_small");
relocation_LCV  = m.addVars(stations,stations,vtype=GRB.INTEGER, lb = 0, name="relocation_large");
##################### reuse the previous model constraints #######################
m.addConstrs((quicksum(relocation[s1, s2] for s2 in stations) <= starting_inventory.loc[s1, 'count'] for s1 in stations));
m.addConstrs(satisfied_demand[s1] <= demand.loc[s1, "count"] for s1 in stations);
m.addConstrs((satisfied_demand[i] <= starting_inventory.loc[i,"count"] \
              + quicksum(relocation[j,i] for j in stations) -quicksum(relocation[i,j] for j in stations) + replenishment.loc[i,"count"]
                  for i in stations),name = "satisfied_demand_vs_inventory_after_rebalancing_and_replenishment");

m.addConstrs(relocation[i,j] == relocation_SCV[i,j] +relocation_LCV[i,j] for i in stations for j in stations);
# either use LCV when bikes >=10 or use SCV when bikes <=9
M = 1e4;
auxiliary = m.addVars(stations,stations,vtype=GRB.BINARY, lb=0, name="relocate 10- SCV or 10+ LCV");
# auxiliary = 1 when use LCV, 0 when use SCV or no relocation
m.addConstrs(relocation_LCV[i,j] <= 0 + M * auxiliary[i, j] for i in stations for j in stations);
m.addConstrs(relocation_LCV[i,j] >= 10 - M * (1 - auxiliary[i, j]) for i in stations for j in stations);
m.addConstrs(relocation_SCV[i,j] <= 9 + M * auxiliary[i, j] for i in stations for j in stations);
m.addConstrs(relocation_SCV[i,j] <= 0 + M * (1 - auxiliary[i, j]) for i in stations for j in stations);
##################### end of reuse  ###############################################

##################### additional constraints  ###############################################
# fixed_cost_yn = 1 when total number of SCV > 0, fixed_cost_yn = 0 when total number of SCV = 0
fixed_cost_yn = m.addVar(vtype=GRB.BINARY, lb=0, name="fixed cost");
m.addConstr(quicksum(relocation_SCV[i,j] for i in stations for j in stations) <= 0 + M * fixed_cost_yn);
m.addConstr(quicksum(relocation_SCV[i,j] for i in stations for j in stations) >= 1 - M * (1 - fixed_cost_yn) );

# relocation_yn[i,j] = 1 when relocation between station i and j is needed , relocation_yn[i,j] = 0 when relocation between station i and j is not needed 
relocation_yn = m.addVars(stations,stations,vtype=GRB.BINARY, lb=0, name="relocation performed or not");
m.addConstrs(relocation[i,j] <= 0 + M * relocation_yn[i, j] for i in stations for j in stations);
m.addConstrs(relocation[i,j] >= 1 - M * (1 - relocation_yn[i, j]) for i in stations for j in stations);

In [44]:
#Insert your code here:
m.setObjective(4*satisfied_demand.sum() # revenue
               -80*fixed_cost_yn # fixed cost if total number of SCV > 0
               -10*(1/3)*quicksum(relocation_yn[i,j] for i in stations for j in stations) # variable cost SCV + LCV
               ,GRB.MAXIMIZE);

**Is the rebalancing operation still marginally profitable?**

In [45]:
# Run the optimization
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nNet revenue: %g' % m.objVal)
    else:
        print('No solution:', m.status)
        
m.optimize()
printSolution()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 3325116 rows, 2374295 columns and 9494422 nonzeros
Model fingerprint: 0x288150e0
Variable types: 0 continuous, 2374295 integer (949443 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [3e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 72024.000000
Presolve removed 1059183 rows and 475177 columns (presolve time = 5s) ...
Presolve removed 1064253 rows and 479191 columns (presolve time = 10s) ...
Presolve removed 1064253 rows and 480247 columns (presolve time = 16s) ...
Presolve removed 1247527 rows and 480247 columns (presolve time = 20s) ...
Presolve removed 1247527 rows and 480247 columns (presolve time = 26s) ...
Presolve removed 1247527 rows and 480247 columns
Presolve time: 25.98s
Presolved: 2077589 rows, 1894048 columns, 713330

*Insert your answer:* 

We can see that the net revenue after introducing procurement (\\$75890) is 0.01\% higher than the cost calculated on per-bike basis (\\$75881.9). However, this marginal increase is very small. 

In [46]:
base_map = generateBaseMap() # generates our base map of NY
# Next we compute the imbalance between demand and inventory
imbalance = np.divide(demand_agg,1+inventory_agg)
imbalance_list = imbalance.fillna(0).reset_index().values.tolist()


# The next loop plots the relocation lines between any two stations.
# The more opaque is the line, the more relocations are made
for i in stations:
    for j in stations:
        if relocation[i,j].x >0:
            p1 = [demand.loc[i,"latitude"],demand.loc[i,"longitude"]]
            p2 = [demand.loc[j,"latitude"],demand.loc[j,"longitude"]]
            opacity = relocation[i,j].x/starting_inventory.loc[i,"count"] +relocation[i,j].x/20
            folium.PolyLine(locations=[p1, p2], color='black', opacity = opacity).add_to(base_map)
            arrows = get_arrows(locations=[p1, p2],color='black', n_arrows=1,opacity = opacity)
            for arrow in arrows:
                arrow.add_to(base_map)

#Plots the heatmap
HeatMap(data= imbalance_list, radius=0,max_zoom=15,minOpacity=0).add_to(base_map)

#Displays the map
base_map    

# Q6. Qualitative Discussion (15 pt)

**Please provide detailed answers to the following questions.**

### Q6.0 
**What are the main limitations of our current modeling approach? What other aspects of the rebalancing would you account for?**

*Insert your answer:* 

There are several main limitations of our current modeling approach. 
1. First, this model doesn't capture the depreciation of bikes. As time goes by, there should be a percentage of bikes that are broken after its life cycle. Therefore, the inventory level of this model is over-estimated. 
2. Second, for some areas (e.g., Midtown), the cost of relocation per bike per km is higher because of traffic jam, which potentially leads to longer labor hours. However, in our model, we neglected this factor and assumes that the unit cost is the same across the whole New York city.  
3. Third, we assume demand is static for a 24-h timeframe. This is too simplistic because the demand of each area can change drastically from time to time within a day. For example, the residential area has higher demand during peak hours (e.g., 8-9am) than other time of the day. 

### Q6.1. 
**In reality, you don't know in advance the exact number of user request in the next hour. How could you remedy this issue?**

*Insert your answer:* 

This issue can be alleviated by using predictive models, using historical hour-by-hour user request volume data together with demographic and economical dimensions, to predict the hourly user request volume. The predicted results of hourly user request volume can then be fed into the optimization model to obtain the optimal way of rebalancing bikes to better match supply and demand. 

### Q6.2.
One of your colleagues has further concerns about how the demand is estimated: *Our historical data only records the customers who are satisfied and complete their trip. Many of our potential customers walk out without even taking a bike!* 

**Explain the above claim. Is this an important issue? Are you currently over-estimating or under-estimating the value of rebalancing? How would you deal with this approach?** 

*Insert your answer:* 

This is an important issue. In our model, we seem to be under-estimating the demand because only satisfied demand is recorded. This is because some potential customers walk to the station without a single bike and walked away. These number of customers is not recorded to historical data. 

I would first go to some areas which are severely under-supplied and count the actual demand level during a certain timeframe (e.g., 1h). Then I would calculate the ratio of actual demand and historical recorded demand. Then I would apply this ratio to all the under-supplied areas in the current demand dataset to arrive at a more realistic demand dataset. Finally, I would feed this new demand dataset into the optimization model. 

### Q6.3. 
In practice, rebalancing is not an easy operation (see the picture below). It requires to hire specialized labor and rent trucks.

<img src="citi-bike-truck.jpg" width="500">

**What other aspects can be incorporated into the optimization model to make it more realistic?**

*Insert your answer:*

1. For now, the distance measurement is not clear. It seems that the dataset is using Euclidean distance to measure the distances. The distance in this dataset is therefore an under-estimation. However, in reality, the distance between stations is higher than Euclidean distance, because of the zig-zag roads or cross-roads. To alleviate this issue, we can connect to google maps API to get the actual distances between stations. 
2. The model can incorporate the bike life cycle, or the depreciation of bikes.  
2. The model can incorporate differentiated cost of relocation by area. This is because the cost of relocation per bike per km tends to be higher because of traffic jam, which potentially leads to longer labor hours. 

### Q6.4 
**For which other operational decisions could Citibike utilize optimization models to improve its efficiency?**

*Insert your answer:* 

1. Citibike can use optimization model to relocate the stations to achieve more residents. 
2. Citibike can use optimization model to select areas with high potential demand to open new stations. 
3. Citibike can use optimization model to determine the optimal price of bike rental. This is because with lower price, there will be higher demand, vice versa. There should be an optimal price level to achieve highest profit (revenue - cost). 

# Executive Summary Report (15 pts)

Write a 1 pager report summarizing your findings (11 pt font, 1 inch margin, pdf/doc format). This report should take the form of an executive summary that combines elements from your analysis and business recommendations. You can also suggest additional analysis that should be conducted to augment the current rebalancing model. Supporting evidence can be provided in the appendix or by referencing the questions of the workshop. The report will be evaluated along 3 dimensions: clarity, scientific validity, and practical relevance.

# Q7. Opening new dock stations (Optional Bonus, 5 pt)

Optimization can be also utilized to decide on where it would be useful to construct new dock locations. Your analysis will be based on the following data sets:

- `demand_extended.csv`: describes the number of user requests in each potential location. 
- `starting_inventory_extended.csv`: describes the initial inventory of bikes at each location (only dock stations have bikes),
- `distances_extended.csv`: describes the distance between any two locations (in km unit).
- `docks_extended.csv`: describes the set of locations that currently hold a dock station, and those where a new dock station can be opened.

**Suppose that you can construct one new dock station to better accommodate the demand. You may assume that users can be routed to any dock station in a radius of 1 km, from their original location. Formulate and implement an integer programming model to decide on the precise locations.**

In [47]:
starting_inventory_extended=pd.read_csv('starting_inventory_extended.csv',index_col=0)
demand_extended=pd.read_csv('demand_extended.csv',index_col=0)
docks_extended =pd.read_csv('docks_extended.csv',index_col=0).set_index("location_id")
distances_extended=pd.read_csv('distances_extended.csv',index_col=0)
distances_extended.columns = list(map(lambda x: int(x),distances_extended.columns.tolist())) # the column names are converted into integers

In [48]:
# Insert your code in the cells below:
